# Machine Learning Model for Subway Dataset

### Imports:

In [17]:
import pandas as pd

### Loading in Weather Data:


### Loading in Subway Data:


In [18]:
turnstile_data=pd.read_csv('cleaned_turnstile_data.csv')
    

In [19]:
turnstile_data.head()

,id,turnstile_id,remote_unit_id,time_stamp,station_name,entries,exits
0,R138 R293 00-03-03 1546290000,R138 R293 00-03-03,R293,1546290000,34 ST-PENN STA,0,0
1,R138 R293 00-02-05 1546290000,R138 R293 00-02-05,R293,1546290000,34 ST-PENN STA,0,0
2,R138 R293 00-02-03 1546290000,R138 R293 00-02-03,R293,1546290000,34 ST-PENN STA,0,0
3,R138 R293 00-00-02 1546290000,R138 R293 00-00-02,R293,1546290000,34 ST-PENN STA,0,0
4,R138 R293 00-03-01 1546290000,R138 R293 00-03-01,R293,1546290000,34 ST-PENN STA,0,0


### Creating busyness percentage for each station
- For each remote unit id and time condense to one row relating to busyness of that station at that time
- The busyness of a place is how many people are currently there, so exits-entries along with the magnitude in the sum of both values, can do both
- Sum of entries and exits for each remote unit id at that time
- Exits - entries for each remote unit id at that time
- Normalize each of these values?

In [56]:
grouped_turnstile_data=turnstile_data.groupby(['remote_unit_id', 'time_stamp']).agg({'entries': 'max', 'exits': 'max'}).reset_index()

grouped_turnstile_data.head()

,remote_unit_id,time_stamp,entries,exits
0,R001,1546300800,0,0
1,R001,1546315200,102,104
2,R001,1546329600,96,51
3,R001,1546344000,260,476
4,R001,1546358400,540,496


In [57]:
len(grouped_turnstile_data)

297994

In [58]:
#add two new columns, one for difference between exits and entries as a proxy for surplus people in that area
#the other for the addition of exits and entries as a proxy for busyness in that area by volume
grouped_turnstile_data['num_people_exited']=grouped_turnstile_data['exits']-grouped_turnstile_data['entries']
#remove negative values and set to zero
grouped_turnstile_data['num_people_exited']=grouped_turnstile_data['num_people_exited'].apply(lambda x: 0 if x<0 else x)

grouped_turnstile_data['sum_exit_entry']=grouped_turnstile_data['exits']+grouped_turnstile_data['entries']

grouped_turnstile_data.head(10)


,remote_unit_id,time_stamp,entries,exits,num_people_exited,sum_exit_entry
0,R001,1546300800,0,0,0,0
1,R001,1546315200,102,104,2,206
2,R001,1546329600,96,51,0,147
3,R001,1546344000,260,476,216,736
4,R001,1546358400,540,496,0,1036
5,R001,1546372800,602,250,0,852
6,R001,1546387200,159,147,0,306
7,R001,1546401600,48,38,0,86
8,R001,1546416000,516,434,0,950
9,R001,1546430400,842,1351,509,2193


In [59]:
#get a standardized z score for each staiton
from sklearn.preprocessing import StandardScaler

#function to be applied to each group
def standardize_group(group):
    scaler = StandardScaler()
    group[['z_score_num_people_exited', 'z_score_sum_exit_entry']] = scaler.fit_transform(group[['num_people_exited', 'sum_exit_entry']])
    return group

#apply the function to each group
grouped_turnstile_data = grouped_turnstile_data.groupby('remote_unit_id', group_keys=True).apply(standardize_group)

grouped_turnstile_data.head(20)


remote_unit_id  time_stamp  entries  exits  \
remote_unit_id                                                 
R001           0            R001  1546300800        0      0   
               1            R001  1546315200      102    104   
               2            R001  1546329600       96     51   
               3            R001  1546344000      260    476   
               4            R001  1546358400      540    496   
               5            R001  1546372800      602    250   
               6            R001  1546387200      159    147   
               7            R001  1546401600       48     38   
               8            R001  1546416000      516    434   
               9            R001  1546430400      842   1351   
               10           R001  1546444800      784    456   
               11           R001  1546459200     1445    686   
               12           R001  1546473600      284    225   
               13           R001  1546488000       68     68   
               14           R001  1546502400      486    418   
               15           R001  1546516800      887   1370   
               16           R001  1546531200      776    482   
               17           R001  1546545600     1526    754   
               18           R001  1546560000      292    243   
               19           R001  1546574400       85     72   

                   num_people_exited  sum_exit_entry  \
remote_unit_id                                         
R001           0                   0               0   
               1                   2             206   
               2                   0             147   
               3                 216             736   
               4                   0            1036   
               5                   0             852   
               6                   0             306   
               7                   0              86   
               8                   0             950   
               9                 509            2193   
               10                  0            1240   
               11                  0            2131   
               12                  0             509   
               13                  0             136   
               14                  0             904   
               15                483            2257   
               16                  0            1258   
               17                  0            2280   
               18                  0             535   
               19                  0             157   

                   z_score_num_people_exited  z_score_sum_exit_entry  
remote_unit_id                                                        
R001           0                   -0.461420               -0.917244  
               1                   -0.449719               -0.687580  
               2                   -0.461420               -0.753358  
               3                    0.802221               -0.096695  
               4                   -0.461420                0.237767  
               5                   -0.461420                0.032630  
               6                   -0.461420               -0.576092  
               7                   -0.461420               -0.821365  
               8                   -0.461420                0.141888  
               9                    2.516327                1.527679  
               10                  -0.461420                0.465202  
               11                  -0.461420                1.458557  
               12                  -0.461420               -0.349772  
               13                  -0.461420               -0.765621  
               14                  -0.461420                0.090604  
               15                   2.364222                1.599031  
               16                  -0.461420                0.485270  
               17          

In [62]:
#convert the z-score to a percentile
from scipy.stats import norm

grouped_turnstile_data['percentile_num_people_exited'] = norm.cdf(grouped_turnstile_data['z_score_num_people_exited'])

grouped_turnstile_data['percentile_sum_exit_entry'] = norm.cdf(grouped_turnstile_data['z_score_sum_exit_entry'])

grouped_turnstile_data.head(10)

remote_unit_id  time_stamp  entries  exits  \
remote_unit_id                                                
R001           0           R001  1546300800        0      0   
               1           R001  1546315200      102    104   
               2           R001  1546329600       96     51   
               3           R001  1546344000      260    476   
               4           R001  1546358400      540    496   
               5           R001  1546372800      602    250   
               6           R001  1546387200      159    147   
               7           R001  1546401600       48     38   
               8           R001  1546416000      516    434   
               9           R001  1546430400      842   1351   

                  num_people_exited  sum_exit_entry  \
remote_unit_id                                        
R001           0                  0               0   
               1                  2             206   
               2                  0             147   
               3                216             736   
               4                  0            1036   
               5                  0             852   
               6                  0             306   
               7                  0              86   
               8                  0             950   
               9                509            2193   

                  z_score_num_people_exited  z_score_sum_exit_entry  \
remote_unit_id                                                        
R001           0                  -0.461420               -0.917244   
               1                  -0.449719               -0.687580   
               2                  -0.461420               -0.753358   
               3                   0.802221               -0.096695   
               4                  -0.461420                0.237767   
               5                  -0.461420                0.032630   
               6                  -0.461420               -0.576092   
               7                  -0.461420               -0.821365   
               8                  -0.461420                0.141888   
               9                   2.516327                1.527679   

                  percentile_num_people_exited  percentile_sum_exit_entry  
remote_unit_id                                                             
R001           0                      0.322249                   0.179507  
               1                      0.326456                   0.245859  
               2                      0.322249                   0.225618  
               3                      0.788788                   0.461484  
               4                      0.322249                   0.593969  
               5                      0.322249                   0.513015  
               6                      0.322249                   0.282276  
               7                      0.322249                   0.205719  
               8                      0.322249                   0.556416  
               9                      0.994071                   0.936704

In [65]:
df_busyness=grouped_turnstile_data[['remote_unit_id', 'time_stamp', 'percentile_num_people_exited','percentile_sum_exit_entry']].copy()
df_busyness.head(10)

remote_unit_id  time_stamp  percentile_num_people_exited  \
remote_unit_id                                                              
R001           0           R001  1546300800                      0.322249   
               1           R001  1546315200                      0.326456   
               2           R001  1546329600                      0.322249   
               3           R001  1546344000                      0.788788   
               4           R001  1546358400                      0.322249   
               5           R001  1546372800                      0.322249   
               6           R001  1546387200                      0.322249   
               7           R001  1546401600                      0.322249   
               8           R001  1546416000                      0.322249   
               9           R001  1546430400                      0.994071   

                  percentile_sum_exit_entry  
remote_unit_id                               
R001           0                   0.179507  
               1                   0.245859  
               2                   0.225618  
               3                   0.461484  
               4                   0.593969  
               5                   0.513015  
               6                   0.282276  
               7                   0.205719  
               8                   0.556416  
               9                   0.936704

### Explain the columns:


### Map the weather to each station for each time interval

In [23]:
'''df_turn['day'] = pd.to_datetime(df_turn['time_stamp'], unit='s').dt.day_name()
df_turn['date'] = pd.to_datetime(df_turn['time_stamp'], unit='s').dt.date
df_turn['time'] = pd.to_datetime(df_turn['time_stamp'], unit='s').dt.time'''

"df_turn['day'] = pd.to_datetime(df_turn['time_stamp'], unit='s').dt.day_name()\ndf_turn['date'] = pd.to_datetime(df_turn['time_stamp'], unit='s').dt.date\ndf_turn['time'] = pd.to_datetime(df_turn['time_stamp'], unit='s').dt.time"